In [214]:
import pandas as pd

df1 = pd.read_json('../data/oglaszamy24.json')
df2 = pd.read_json('../data/fb.json')
df3 = pd.read_json('../data/olx_remote_work.json')

df = pd.concat([df1,df2,df3])
df

,id,text
0,http://krakow.oglaszamy24.pl/ogloszenie/154952...,"Witam, tak jak w tytule poszukuję kobiete do s..."
1,http://opole.oglaszamy24.pl/ogloszenie/1885440...,"Chcesz zbudować, odnowić, przywrócić lub po pr..."
2,http://warszawa.oglaszamy24.pl/ogloszenie/1017...,"Medycyna pracy od ręki ,badania wysokościowe,\..."
3,http://lodz.oglaszamy24.pl/ogloszenie/49325871...,"Medycyna pracy od ręki ,badania wysokościowe,\..."
4,http://katowice.oglaszamy24.pl/ogloszenie/1330...,"Medycyna pracy od ręki ,badania wysokościowe,\..."
...,...,...
1344,http://www.olx.pl/oferta/praca/tworzenie-grafi...,OGŁOSZENIE JEST NADAL AKTUALNE - NATOMIAST LIC...
1345,http://www.olx.pl/oferta/praca/tworzenie-grafi...,OGŁOSZENIE JEST NADAL AKTUALNE - NATOMIAST LIC...
1346,http://www.olx.pl/oferta/praca/latwa-dodatkowa...,"Szukamy osób do prostej pracy zdalnej, wykonyw..."
1347,http://www.olx.pl/oferta/praca/doradca-ubezpie...,Zadania na tym stanowisku:Nawiązywanie relacji...


In [215]:
df['text'] = df['text'].apply(lambda x: x.lower() if x is not None else '')

In [216]:
df.drop(['id'], axis=1, inplace=True)
df

,text
0,"witam, tak jak w tytule poszukuję kobiete do s..."
1,"chcesz zbudować, odnowić, przywrócić lub po pr..."
2,"medycyna pracy od ręki ,badania wysokościowe,\..."
3,"medycyna pracy od ręki ,badania wysokościowe,\..."
4,"medycyna pracy od ręki ,badania wysokościowe,\..."
...,...
1344,ogłoszenie jest nadal aktualne - natomiast lic...
1345,ogłoszenie jest nadal aktualne - natomiast lic...
1346,"szukamy osób do prostej pracy zdalnej, wykonyw..."
1347,zadania na tym stanowisku:nawiązywanie relacji...


In [217]:
vocab = []
with open('../data/suspicious_phrases.txt') as f:
    for line in f.readlines():
        vocab.append(line.strip())

vocab = pd.DataFrame(vocab, columns=['text'])
vocab

,text
0,sprzedam konto
1,kupię konto
2,zapłacę za założenie konta
3,konto bankowe bez komornika
4,udostępnię konto bankowe
5,czyste konto bankowe
6,bezpieczne konto bankowe
7,przelewy za %
8,procent od kwoty przelewu
9,konto na słupa


In [218]:
df_with_phrases = pd.concat([df, vocab])
df_with_phrases

,text
0,"witam, tak jak w tytule poszukuję kobiete do s..."
1,"chcesz zbudować, odnowić, przywrócić lub po pr..."
2,"medycyna pracy od ręki ,badania wysokościowe,\..."
3,"medycyna pracy od ręki ,badania wysokościowe,\..."
4,"medycyna pracy od ręki ,badania wysokościowe,\..."
...,...
33,gwarancja zadowolenia
34,czyjeś konto
35,do legalnych rzeczy
36,legalne


In [219]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf = TfidfVectorizer(ngram_range=(1,5), min_df=10)
# tfidf_data = tfidf.fit_transform(df_with_phrases['text'])
# tfidf.get_feature_names_out()

In [220]:
# tfidf_matrix = pd.DataFrame(tfidf_data.todense(), columns = tfidf.get_feature_names_out()).astype('float32')
# tfidf_matrix

In [221]:
df_with_phrases.shape

(3395, 1)

In [223]:
from nltk import ngrams
import re

def get_ngrams(offer):
    offer_ngrams_list = []
    offer_clean = re.sub('[\W]+', ' ', offer)
    offer_list = offer_clean.split()
    for i in range(2,6):
        offer_ngrams = ngrams(offer_list, i)
        for gram in offer_ngrams:
            offer_ngrams_list.append(gram)
    return offer_ngrams_list

df_with_phrases['ngrams'] = df_with_phrases['text'].apply(get_ngrams)

In [224]:
ngram_duplicates = pd.Series()
for ngrams in df_with_phrases['ngrams']:
    g = pd.Series(ngrams).value_counts()
    ngram_duplicates = pd.concat([ngram_duplicates,g[g>1]])

In [225]:
data = {'keys' : list(ngram_duplicates.index),
        'values' : ngram_duplicates.values}

ngram_duplicates = pd.DataFrame(data=data)

In [226]:
# !python -m spacy download pl_core_news_lg

In [227]:
import spacy

nlp_pl = spacy.load("pl_core_news_lg")
stopwords = nlp_pl.Defaults.stop_words

In [228]:
stopwords.update(['facebook', 'www', 'https', 'bit', 'ly', 'com', 'groups'])
stopwords

{'a',
 'aby',
 'ach',
 'acz',
 'aczkolwiek',
 'aj',
 'albo',
 'ale',
 'alez',
 'ależ',
 'ani',
 'az',
 'aż',
 'bardziej',
 'bardzo',
 'beda',
 'bede',
 'bedzie',
 'bez',
 'bit',
 'bo',
 'bowiem',
 'by',
 'byc',
 'byl',
 'byla',
 'byli',
 'bylo',
 'byly',
 'bym',
 'bynajmniej',
 'być',
 'był',
 'była',
 'było',
 'były',
 'będzie',
 'będą',
 'będę',
 'cala',
 'cali',
 'caly',
 'cała',
 'cały',
 'chce',
 'choć',
 'ci',
 'cie',
 'ciebie',
 'cię',
 'co',
 'cokolwiek',
 'com',
 'coraz',
 'cos',
 'coś',
 'czasami',
 'czasem',
 'czemu',
 'czy',
 'czyli',
 'często',
 'daleko',
 'dla',
 'dlaczego',
 'dlatego',
 'do',
 'dobrze',
 'dokad',
 'dokąd',
 'dosc',
 'dość',
 'duzo',
 'dużo',
 'dwa',
 'dwaj',
 'dwie',
 'dwoje',
 'dzis',
 'dzisiaj',
 'dziś',
 'facebook',
 'gdy',
 'gdyby',
 'gdyz',
 'gdyż',
 'gdzie',
 'gdziekolwiek',
 'gdzies',
 'gdzieś',
 'go',
 'godz',
 'groups',
 'https',
 'i',
 'ich',
 'ile',
 'im',
 'inna',
 'inne',
 'inny',
 'innych',
 'iv',
 'ix',
 'iz',
 'iż',
 'ja',
 'jak',
 'jakas

In [229]:
ngram_duplicates['stopwords'] = ngram_duplicates['keys'].apply(lambda x : 1 if any([word in x for word in stopwords]) else 0)

In [230]:
ngram_duplicates = ngram_duplicates[(ngram_duplicates['stopwords']==0) & (ngram_duplicates['values']>2)].sort_values(by=['values'], ascending=False).drop_duplicates(subset=['keys'])
ngram_duplicates.to_csv('../data/potential_phrases.csv')

In [231]:
ngram_duplicates

,keys,values,stopwords
11189,"(803, 803)",94,0
11190,"(803, 803, 803)",86,0
11191,"(803, 803, 803, 803)",80,0
11192,"(803, 803, 803, 803, 803)",74,0
11401,"(jobb, her)",62,0
...,...,...,...
6728,"(serwis, sprzątający)",3,0
5870,"(0, 00)",3,0
5768,"(koło, wrocławia)",3,0
5763,"(ce, driver)",3,0
